# 개별 미션 풀이 

- '37_Data.xlsx'를 불러와 아래의 분석을 실시 
- 해당 데이터는 모 카드사에서 고객들이 전화상담을 진행하고 나온 고객 VOC 데이터 이다. 

In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
import scipy.stats as stats

In [2]:
df1 = pd.read_excel('37_Data.xlsx')
print(df1.shape)
df1.head()

(1000, 4)


,NO,분류,VOC,대응우선순위
0,1,불만>고객서비스>상담시스템,상담직원과 빠른연결했슴좋겠어요,5
1,2,불만>고객서비스>상담시스템,안기다리고. 빨리상담되었음좋겠다,5
2,3,불만>고객서비스>상담시스템,너무 오래기다리고 상담이 늦어져요,5
3,4,불만>고객서비스>상담시스템,상담직원과 통화가 매우 힘들다,5
4,5,불만>고객서비스>상담시스템,상담받으려면오래기다리는게불편함,5


1.  '분류'에서 '불만'과'칭찬' VOC에 대해, 고객 VOC를 입력했을 때, 해당 VOC가 '불만'에 해당하는 것인지 '칭찬'에 해당하는 것인지 분류하는 모델을 RNN 계열의 모델을 선택해 생성하시오. 

In [7]:
def func1(row):
    return row.split('>')[0]

df1['Target'] = df1['분류'].apply(func1)
df1['Target'].value_counts()

Target
불만    512
칭찬    481
중립      7
Name: count, dtype: int64

In [9]:
cond1 = df1['Target'].isin(['불만','칭찬'])
df2 = df1.loc[cond1]

In [10]:
Y = df2['Target'].replace({'불만':0,'칭찬':1})
X = df2[['VOC']]

In [26]:
df_stopword   = pd.read_csv('stopword.txt')
stopword_list = df_stopword['불용어'].tolist()

In [2]:
from  konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
okt = Okt()

In [29]:
# 텍스트 처리 함수 
def text_preprocessing(X):
    sent1 = [[y for y in okt.nouns(x) if y not in stopword_list] for x in X['VOC'].tolist()]
    sent_series = pd.Series(sent1)
    token_model = Tokenizer()
    token_model.fit_on_texts(sent_series)
    se1 = token_model.texts_to_sequences(sent_series)
    return pad_sequences(se1, maxlen=15)

In [31]:
# text_preprocessing(X)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train,X_test,Y_train,Y_test =train_test_split(X,Y, 
                                                test_size=0.3,random_state=1234)

In [35]:
X_train_NN = text_preprocessing(X_train)
X_test_NN  = text_preprocessing(X_test)

In [3]:
from keras.layers import GRU
from keras.layers import Embedding
from keras.layers import Dense
from keras.models import Sequential

In [38]:
model_gru = Sequential()
model_gru.add(Embedding(10000, 8))
model_gru.add(GRU(128))
model_gru.add(Dense(1, activation='sigmoid'))
model_gru.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['accuracy'])
model_gru.fit(X_train_NN, Y_train, epochs=10, 
              batch_size=32, validation_split=0.2)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.4456 - loss: 0.6926 - val_accuracy: 0.4964 - val_loss: 0.6824
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.5504 - loss: 0.6630 - val_accuracy: 0.6978 - val_loss: 0.6338
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.7390 - loss: 0.5847 - val_accuracy: 0.7482 - val_loss: 0.5337
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7283 - loss: 0.5223 - val_accuracy: 0.7410 - val_loss: 0.5499
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.7961 - loss: 0.4358 - val_accuracy: 0.7626 - val_loss: 0.5245
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8381 - loss: 0.3984 - val_accuracy: 0.7626 - val_loss: 0.4687
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8464 - loss: 0.3592 - val_accuracy: 0.7626 - val_loss: 0.4853
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.8945 - loss: 0.3289 - val_accuracy: 0.7770 - 

In [39]:
def threshold_func(result):
    if result >= 0.5:
        return 1 
    return 0 

In [43]:
Y_train_pred = pd.Series(model_gru.predict(X_train_NN).flatten()).apply(threshold_func)
Y_test_pred = pd.Series(model_gru.predict(X_test_NN).flatten()).apply(threshold_func)

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89       343
           1       0.89      0.91      0.90       352

    accuracy                           0.90       695
   macro avg       0.90      0.90      0.90       695
weighted avg       0.90      0.90      0.90       695



In [46]:
print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77       169
           1       0.70      0.66      0.68       129

    accuracy                           0.73       298
   macro avg       0.73      0.72      0.72       298
weighted avg       0.73      0.73      0.73       298



2. 생성한 모델에 아래의 VOC를 입력해 해당 값이 어디로 분류되는지 확인 

In [47]:
input_text = '''대출 상환을 앱에서 쉽게 할 수 있도록 보완 해주세요. 
상환이 너무 어려워 앱 쓰다가 정말 짜증나요'''

In [52]:
input_data = pd.DataFrame(data=[input_text], columns=['VOC'])

In [57]:
result1 = model_gru.predict(text_preprocessing(input_data)).flatten()
pd.Series(result1).apply(threshold_func).replace({1:'칭찬',0:'불만'})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


0    칭찬
dtype: object

3. '분류'에서 DS카드에 해당하는 VOC들만 뽑아, '무이자할부'라는 단어가 등장할 때, 가장 유사도가 높은 단어들을 확인하시오.

In [59]:
def func2(row):
    if 'DS카드' in row:
        return'카드서비스'
    return '일반'

df1['VOC유형'] = df1['분류'].apply(func2)
df1['VOC유형'].value_counts()

VOC유형
일반       744
카드서비스    256
Name: count, dtype: int64

In [62]:
cond1 = (df1['VOC유형']=='카드서비스')
df3 = df1.loc[cond1]

In [66]:
text1 = [[y for y in okt.nouns(x) if y not in stopword_list] for x in df3['VOC'].tolist()]

In [71]:
# 무이자 할부 단어를 하나의 단어로 묶어서 처리 
text2 = [ ]

for k in text1:
    # 현재 처리중인 문장에서 변환된 토큰값을 저장 
    sent_list = [ ]
    # 현재 처리중인 토큰 번호 
    sent_num = 0 
    while sent_num < len(k) : # 문장의 길이만큼 반복해서 수행 
        # 3가지 토큰 (무/ 이자 / 할부)을 가져와 처리 
        # 문장 내 토큰이 3개 이상인 경우 
        # '무' '이자' '할부' 라는 토큰이 연속적으로 나오는 경우
        if (sent_num + 2  < len(k))and(k[sent_num] =="무")and(k[sent_num + 1] =="이자")and(k[sent_num + 2] =="할부"): 
            # '무이자할부'를 추가한 뒤 3개 토큰을 넘어 처리 
            sent_list.append('무이자할부')
            sent_num += 3 
        else : # 토큰이 존재 하지 않아 다음 토큰을 확인 
            sent_list.append(k[sent_num])
            sent_num += 1 
    
    text2.append(sent_list)

In [73]:
from gensim.models import Word2Vec

In [74]:
model_sg = Word2Vec(text2, vector_size=100, min_count=1, window=3 ,sg=1)

In [75]:
model_sg.wv.most_similar('무이자할부')

[('지속', 0.32144346833229065),
 ('회비', 0.2649388611316681),
 ('만족', 0.23439793288707733),
 ('권유', 0.22731447219848633),
 ('형식', 0.22131164371967316),
 ('누구', 0.21935860812664032),
 ('왜', 0.2085740566253662),
 ('출생', 0.20226159691810608),
 ('먾', 0.1935790628194809),
 ('제일', 0.19019554555416107)]

4. 고객 '대응우선순위' 10위 권 내 VOC들에 대해 '토픽 모델링'을 실시하여, 주제문장 3문장을 확인하시오. 

In [80]:
cond1 = (df1['대응우선순위'] <= 10)
sent1 = df1.loc[cond1]['VOC']

In [81]:
t3=[[y for y in okt.nouns(x) if y not in stopword_list] for x in sent1.tolist()]

In [83]:
from gensim import corpora
from gensim.models import LdaModel

In [88]:
dict1 = corpora.Dictionary(t3)
corpus = [dict1.doc2bow(x) for x in t3]

In [90]:
# LDA Modeling 
lda_model = LdaModel(corpus, num_topics=3, id2word=dict1, passes=20)

In [91]:
lda_model.print_topics()

[(0,
  '0.127*"상담" + 0.118*"감사" + 0.047*"친절" + 0.034*"설명" + 0.033*"답변" + 0.032*"고객" + 0.022*"질문" + 0.022*"기분" + 0.018*"전화" + 0.018*"말투"'),
 (1,
  '0.134*"시간" + 0.106*"통화" + 0.095*"연결" + 0.082*"상담" + 0.071*"대기" + 0.030*"응대" + 0.025*"직원" + 0.022*"상담사" + 0.012*"고객" + 0.011*"느낌"'),
 (2,
  '0.266*"상담" + 0.054*"친절" + 0.024*"직원" + 0.022*"처리" + 0.017*"번" + 0.017*"고객" + 0.015*"내용" + 0.012*"회원" + 0.011*"골드" + 0.011*"매우"')]

5. '대응우선순위'가 10위권 내 VOC와 아닌 VOC 에 대해 고객 '불만/칭찬' 여부의 유의미한 차이가 있는지 가설검정을 실시하시오. 

In [93]:
cond1 = (df1['대응우선순위']<=10)
df1.loc[cond1, '상위권여부'] = '우선순위'
df1.loc[~cond1, '상위권여부'] = '후순위'

In [94]:
df1['상위권여부'].value_counts()

상위권여부
우선순위    603
후순위     397
Name: count, dtype: int64

In [97]:
# 귀무가설 : 상위권 여부에 따라 불만/칭찬 비율의 차이가 없다. (독립이다.)
# 대립가설 : 상위권 여부에 따라 불만/칭찬 비율의 차이가 있다. (연관이다.)
c1 = pd.crosstab(df1['상위권여부'],df1['Target'])
c2 = c1[['불만','칭찬']]
c2

Target,불만,칭찬
상위권여부,,
우선순위,281,322
후순위,231,159


In [98]:
stats.chi2_contingency(c2)
# P.value < 0.05 / 대립가설 참 / 상위권 여부에 따라 불만/칭찬 비율의 차이가 있다. 

Chi2ContingencyResult(statistic=14.625520255291462, pvalue=0.00013112677331605097, dof=1, expected_freq=array([[310.91238671, 292.08761329],
       [201.08761329, 188.91238671]]))

# Attention Mechansim 

- Seq2Seq 모델의 단점 :
     1. Context Vector의 고정되 Matrix 크기로 인해 정보 손실이 발생 
     2. 긴 Sequence에 대해 RNN 계열 모델의 기울기 소실 문제가 발생 
![image1](https://i0.wp.com/blog.kakaocdn.net/dn/RtMGM/btrXyiWoa10/5zt5t6BgqEXrtsU13OpHdK/img.png?w=900&ssl=1)


- Attention Mechansim: Decoder에서 출력 단어를 예측하는 매 시점마다, Encoder에서 전체 입력문장을 다시한번 참고하여 출력 
- 단어를 출력하는 특정 시점에서 예측해야할 단어와 연관이 있는 입력단어를 집중(Attention)하여 계산 
- 원리 : 
    - 모든 단어 쌍에 대한 Key - Value 형태의 유사도 분석을 수행 
    - 각 단어 쌍의 유사도를 이용하여 Attension Score 계산 
    - Attension Score : 특정 디코더 시점 t에서 단어를 예측하기 위해, 인코더의 모든 Hidden State 상태 각각이 디코더의 현시점의 Hidden State상태와 얼마나 유사한지를 비교 
    - Attension Score를 활용해 Attension 분포를 추정 
    - 추정된 분포는 출력단계에서 좀 더 유사한 단어 쌍이 생성되도록 유도 

In [4]:
import pickle

In [101]:
# 영어 문장과 한국어 문장을 One Hot Encoding 실시한 결과 
load_seq_data = pickle.load(open('eng_to_kor.sav','rb'))
encoder_input_data, decoder_input_data, decoder_target_data = load_seq_data

In [102]:
# 단어 토큰에 대한 사전 
token_index = pickle.load(open('token_index.sav','rb'))
input_token_index, target_token_index = token_index

In [104]:
num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)

In [105]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Attention

In [106]:
# 학습 Parameter 지정 
batch_size = 32 # (64이상)
epochs = 20 # (200~500이상)
node_num = 64 # (1024 이상)
max_encoder_seq_length = 39 
max_decoder_seq_length = 38 

In [108]:
# 인코더 / 디코더 정의 
encoder_input = Input(shape=(None, num_encoder_tokens))
# Attension Score를 계산하기 위해, Encoder에서도 각 Node의 결과를 출력 
# return_sequences=True
encoder_lstm  = LSTM(node_num, return_state=True, return_sequences=True)
encoder_output, state_h, state_c = encoder_lstm(encoder_input)
encoder_state = [state_h, state_c]
decoder_input = Input(shape=(None, num_decoder_tokens))
decoder_lstm  = LSTM(node_num, return_state=True, return_sequences=True) 
decoder_output, _, _ = decoder_lstm(decoder_input, initial_state = encoder_state)

# 어탠션 레이어를 추가 
attention = Attention()
attention_output = attention([decoder_output,encoder_output])
# 최종 출력을 계산 
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(attention_output)

# 모델 정의 
model = Model( [encoder_input, decoder_input], decoder_outputs )

In [ ]:
# 학습 실시 
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,epochs=epochs, validation_split=0.2)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 202ms/step - loss: 2.0222 - val_loss: 2.4391
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 171ms/step - loss: 1.6527 - val_loss: 2.4379
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 161ms/step - loss: 1.6419 - val_loss: 2.4379
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 161ms/step - loss: 1.6370 - val_loss: 2.4361
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 147ms/step - loss: 1.6294 - val_loss: 2.4364
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 94ms/step - loss: 1.6542 - val_loss: 2.4368
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 1.6340 - val_loss: 2.4358
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 1.6328 - val_loss: 2.4334
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - loss: 1.6385 - val_loss: 2.4344
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 1.6370 - val_loss: 2.4372
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 1.6489 - val_loss: 2.4356
Epoch 12/20
125/125 ━━━━━━━━━━

In [5]:
# 자연어 생성
encoder_model = Model(encoder_input, encoder_state)

decoder_state_input_h = Input(shape=(node_num, )) # Hidden State 초기화
decoder_state_input_c = Input(shape=(node_num, )) # Cell State 초기화
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_input,
                                                 initial_state=decoder_state_inputs)

decoder_outputs = decoder_dense(decoder_outputs)



NameError: name 'Model' is not defined

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq,verbose=0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value,verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence